In [1]:
import os
import requests
os.environ['x-app-id'] = 'YOUR_APP_ID_HERE'
os.environ['x-app-key'] = 'YOUR_APP_KEY_HERE'

# 現在你可以使用這些環境變數
print(os.environ['x-app-id'])
print(os.environ['x-app-key'])

YOUR_APP_ID_HERE
YOUR_APP_KEY_HERE


In [16]:
headers = {
    "Content-Type": "application/json",
    "x-app-id": os.getenv("NUTRITIONNIX_APP_ID"),
    "x-app-key": os.getenv("NUTRITIONNIX_APP_KEY")
}

def get_nutrition_with_nlp(headers, query):
    """
    Retrieves nutritional information for foods using natural language processing from the Nutritionix API.

    Parameters:
        headers (dict): Headers containing authentication information required by the API.
        query (str): The user's natural language query describing the food for which they want to obtain nutritional information.

    Returns:
        list: A list of dictionaries containing filtered nutritional information. Each dictionary represents the nutritional information for one food item.

    Exceptions:
        Returns an error message string if the request fails or if an exception occurs during the API call.
    """
    url = "https://trackapi.nutritionix.com/v2/natural/nutrients"

    payload = {
        "query": query
    }
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # if error happened, raise the error status
        saved_response = []
        # only keep important nutrition info to minimize the token
        for food in response.json()["foods"]:
            keys_to_remove = set(['full_nutrients', 'nix_brand_name', 'nix_brand_id', 'nix_item_name', 'nix_item_id', 'upc', 'consumed_at', 'source', 'ndb_no', 'lat', 'lng', 'meal_type', 'photo', 'sub_recipe', 'class_code', 'brick_code', 'tag_id'])
            saved_food_info = {key: value for key, value in food.items() if key not in keys_to_remove}
            saved_response.append(saved_food_info)
        return saved_response
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"
    
def find_restaurant_food(headers, query):
    """
    Searches for branded foods in the Nutritionix API based on the search query.

    Parameters:
        headers (dict): Headers containing the authentication details required by the API.
        query (str): The search query used to find foods, which can include food names, brands, etc.

    Returns:
        list: A list of dictionaries containing the search results. Each dictionary includes key information such as the food name and brand name.

    Exceptions:
        Returns a dictionary with an error message if the request fails or if an exception occurs during the API call.
    """
    url = f"https://trackapi.nutritionix.com/v2/search/instant/?query={query}"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # if error happened, raise the error status
        branded_food = response.json()["branded"]
        saved_response = []
        # only keep important nutrition info to minimize the token
        for food in branded_food:
            keys_to_save = set(['food_name', 'brand_name_item_name', 'brand_name'])
            saved_food_info = {key: value for key, value in food.items() if key in keys_to_save}
            saved_response.append(saved_food_info)
        return saved_response
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

In [ ]:
def search_meal_by_name(query):
    """
    Searches for meal information by name using TheMealDB API.

    Parameters:
        query (str): The name of the meal to search for. This can be a full name or a partial name, as the API supports searching with partial matches.

    Returns:
        dict: A dictionary containing the API's response. This includes an array of meals that match the search query, with each meal containing details such as its name, category, area, instructions, and ingredients.

    Exceptions:
        Returns an error message string if the request encounters an error or if an exception occurs during the API call. This could be due to various reasons such as network issues, invalid API keys, or errors from TheMealDB API.

    Notes:
        The function uses a GET request to retrieve data from TheMealDB API based on the search query provided. The 'API-key' in the headers is a placeholder and should be replaced with a valid API key if required by TheMealDB API.
    """
    headers = {
        "Content-Type": "application/json",
        "API-key" : "1"
    }

    url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={query}"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # if error happened, raise the error status
        return response.json()
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

In [13]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

class GoogleSearch(BaseModel):
    query: str = Field(description="search query")
google_search_tool = StructuredTool.from_function(
        func=GoogleSearchAPIWrapper().run,
        name="google_search",
        description="Search Google for related results",
        args_schema=GoogleSearch
    )

In [14]:
google_search_tool.run("牛肉麵推薦")

'Jan 8, 2024 ... 想知道台北有哪些推薦必吃的牛肉麵嗎？Klook編輯精選台北9家最受好評的必吃牛肉麵，包括入選台北米其林指南的林東芳牛肉麵，或是台北牛肉麵比賽冠軍的\xa0... Aug 11, 2023 ... 台北牛肉麵霸主PK！4星起跳8大名店：70元開吃、米其林推薦、牛肉麵節冠軍 · 半島牛肉麵：台北牛肉麵節冠軍 · 阿郎牛肉麵：內行才知70元開吃 · 天下三絕\xa0... Aug 22, 2023 ... 牛肉麵都有米芝蓮？來自高雄的小王牛肉麵，曾於台北國際牛肉麵節獲雙料冠軍，更是2014及15年米芝蓮推介食肆。牛骨湯中加入清宮御膳秘方的中藥材熬製而成，\xa0... Aug 24, 2023 ... 台北牛肉麵霸主PK！4星起跳8大名店：70元開吃、米其林推薦、牛肉麵節冠軍 · △「半島牛肉麵」曾榮獲2018年台北牛肉麵節的冠軍殊榮。 · △「半筋半肉湯」\xa0... Mar 19, 2023 ... 【台北】20+ 大台北牛肉麵推薦，紅燒牛肉麵、清燉牛肉麵、蕃茄牛肉麵、椒麻牛肉麵，台北牛肉麵推薦 · 太官私房牛肉麵. 【台北】後山埤｜太官私房牛肉麵，\xa0... Mar 4, 2023 ... 【台湾】五大必吃的牛肉面店推荐：永康街、金春发都有，哪一家最好吃？ · 台湾牛肉面推荐一：永康牛肉面 · 台湾牛肉面推荐二：洪师父牛肉面 · 台湾牛肉面\xa0... Dec 30, 2023 ... 2024台北牛肉麵推薦｜必吃8間牛肉麵店，香濃湯頭、Q彈肉質暖心又暖胃！ · 2023 台北國際牛肉麵節得獎名單 · 紅燒組 · 清燉組 · 台北牛肉麵推薦：中正區. May 9, 2023 ... 【台灣】必試5大台北牛肉麵推薦天下三絕/洪師父/金春發/清真/東館 · 台北牛肉麵推薦一：天下三絕麵食館 · 台北牛肉麵推薦二：洪師父牛肉麵 · 台北牛肉麵\xa0... 2022牛肉麵推薦》老饕淚推全台最好吃的17間神店！200元有找就能享用平民五星級美味 · 1. 金春發》台北 · 2. 穆記牛肉麵》台北 · 3. 太官牛肉麵》台北 · 4. 中國清真牛肉\xa0... 所有分類 · 珍苑紅燒半筋牛肉麵3盒組【常溫】. 已售完. 珍苑紅燒半筋牛肉麵3盒組【常溫】. $660. $507 · 珍苑紅燒牛三寶牛肉麵3盒組【常溫】. 已售完. 珍苑